In [1]:
from generator import *
from shredder import *
from fitter import *
from exec_tools import run
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from joblib import Parallel, delayed
from math import sqrt
import time
import itertools
#np.random.seed(10)



/Users/eilifm/anaconda3/envs/rsds/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data_gen_dict = config_maker(2, 10, [.5], [[1,2]],[5])
data_gen_dict = config_maker(4, 10, [1,1,1], [],[])

In [3]:
estimates = list()
in_ci = list()
for iters in range(100):
    fit_data, test_data, true_coeffs = generate_ind_model(
                                            data_gen_dict["dist_list"],
                                            data_gen_dict["main_coeffs"],
                                            data_gen_dict["interaction_coeffs"],
                                            data_gen_dict["interactions"],
                                            intercept=10,
                                            n=100,
                                            beta_sigma=0
                                        )

    tmp_data = fit_data.copy(deep=True)
    tmp_data = uniform_shred_cols(['x1'], .5, tmp_data)
    tmp_data = tmp_data.dropna()
    model, metrics = fit_lm(tmp_data, test_data)
    b_estimate_results = beta_target_check(metrics, true_coeffs, as_dataframe=False)
    in_ci.append([b_estimate_results[key]['in_ci'] for key in b_estimate_results.keys()])
    
    estimates.append(model.params.values)

estimates_df = pd.DataFrame(estimates, columns=model.params.index)
in_ci_df = pd.DataFrame(in_ci)

In [4]:
print(estimates_df.mean())
print(in_ci_df.mean())

const    10.0
x1       10.0
x2       10.0
x3       10.0
x4       10.0
dtype: float64
0    0.81
1    0.74
2    0.85
3    0.87
4    0.90
dtype: float64


In [5]:
tmp_data = fit_data.copy(deep=True)
tmp_data = uniform_shred_cols(['x1'], 0, tmp_data)

test = pd.DataFrame(np.array([tmp_data['x1'], fit_data['x1']]).T, columns=['x1_altered', 'x1_original'])
print(test)
test[['x1_altered', 'x1_original']].assign(NE=test.x1_altered == test.x1_original)

    x1_altered  x1_original
0     0.522054     0.522054
1     0.879428     0.879428
2     0.762485     0.762485
3     0.864024     0.864024
4     0.722210     0.722210
5     0.531083     0.531083
6     0.613244     0.613244
7     0.629536     0.629536
8     0.959190     0.959190
9     0.177162     0.177162
10    0.597171     0.597171
11    0.614091     0.614091
12    0.732349     0.732349
13    0.893135     0.893135
14    0.475235     0.475235
15    0.255864     0.255864
16    0.674234     0.674234
17    0.096581     0.096581
18    0.210147     0.210147
19    0.186248     0.186248
20    0.340971     0.340971
21    0.683949     0.683949
22    0.424472     0.424472
23    0.508685     0.508685
24    0.511914     0.511914
25    0.715462     0.715462
26    0.874424     0.874424
27    0.123111     0.123111
28    0.412546     0.412546
29    0.965554     0.965554
..         ...          ...
70    0.839825     0.839825
71    0.585306     0.585306
72    0.650935     0.650935
73    0.255413     0

,x1_altered,x1_original,NE
0,0.522054,0.522054,True
1,0.879428,0.879428,True
2,0.762485,0.762485,True
3,0.864024,0.864024,True
4,0.722210,0.722210,True
5,0.531083,0.531083,True
6,0.613244,0.613244,True
7,0.629536,0.629536,True
8,0.959190,0.959190,True
9,0.177162,0.177162,True


In [6]:
print(true_coeffs)
print(model.summary())


{'x1': 10, 'x2': 10, 'x3': 10, 'x4': 10, 'const': 10}
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 9.403e+30
Date:                Fri, 02 Mar 2018   Prob (F-statistic):               0.00
Time:                        14:56:06   Log-Likelihood:                 1576.0
No. Observations:                  50   AIC:                            -3142.
Df Residuals:                      45   BIC:                            -3133.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

In [7]:
model.params.values


array([ 10.,  10.,  10.,  10.,  10.])

# Rewrite of the Invert 
It seems that it still only works to invert if only one column is missing. If more than one is missing then you're still out of luck. 

In [400]:
#data_gen_dict = config_maker(3, 5, [.4, .4], [[1,2], [1,3], [2,3]],[5, 10, 8])
data_gen_dict = config_maker(
    2,
    5,
    [.4],
    [
        [1,2]
    ], 
    [4])
#data_gen_dict = config_maker(1, 10, [], [],[])
fit_data, test_data, true_coeffs = generate_ind_model(
                                        data_gen_dict["dist_list"],
                                        data_gen_dict["main_coeffs"],
                                        data_gen_dict["interaction_coeffs"],
                                        data_gen_dict["interactions"],
                                        intercept=1,
                                        n=100,
                                        beta_sigma=0
                                    )

# Drop some data
#tmp_data = uniform_shred_cols(['x2'], .5, fit_data)
#tmp_data = fit_data.copy()
#tmp_data = uniform_shred_cols(['x2'], .2, tmp_data)



In [401]:
fit_data.head()


,x1,x2,x1:x2,y
0,0.645931,0.247720,0.160010,5.365135
1,0.223713,0.279355,0.062495,2.927256
2,0.499990,0.486799,0.243395,5.447128
3,0.484287,0.583826,0.282739,5.720043
4,0.119905,0.819088,0.098213,3.630554


In [402]:
tmp_data

,x1,x2,x1:x2,y
0,0.255508,0.213647,0.054589,2.717112
1,0.923035,0.434936,0.401461,8.945513
2,0.317330,0.862168,0.273592,5.874651
3,0.552253,0.461475,0.254851,5.281028
4,0.698121,0.993329,0.693464,8.900333
5,0.510244,0.315453,0.160958,4.987083
6,0.792447,0.566210,0.448692,7.769159
7,0.554047,0.784109,0.434433,5.965653
8,0.845804,0.608386,0.514576,8.225612
9,0.498972,0.304153,0.151764,5.063671


In [403]:
import importlib
import fixer
importlib.reload(fixer)




<module 'fixer' from '/Users/eilifm/github/eilifm/rit_missing_data/fixer.py'>

In [404]:
#tmp_data = uniform_shred_cols(['x1'], .2, fit_data.copy())
tmp_data=fit_data.copy()
fit_data.columns.values
# Find the invertable rows
# Find rows where nothing but the target is missing


array(['x1', 'x2', 'x1:x2', 'y'], dtype=object)

In [407]:
inverted,  fitted = fixer.olsinv_singlex(tmp_data, 'x1', test_mode=True)
print(inverted)
print(np.round(tmp_data.loc[inverted.index, 'x1'].values, 8))

(np.round(tmp_data.loc[inverted.index, 'x1'].values, 8) == np.round(inverted.values, 8)).all()

print(fitted.params)

# print(str.format('{0:.35f}', tmp_data.loc[inverted.index, 'x1'].values[0]))
# print(str.format('{0:.35f}', inverted.values[0]))

# target = 'x1'
# fig = sm.graphics.abline_plot(slope=fitted.params['x1'], intercept=fitted.params['const'])
# ax = fig.axes[0]
# ax.scatter(tmp_data['x1'], tmp_data['y'])
print(fit_data.shape)

83    0.572979
30    0.512143
56    0.376648
24    0.046133
16    0.532484
23    0.858210
2     0.499990
27    0.739140
28    0.579917
13    0.051032
Name: y, dtype: float64
[ 0.5729795   0.51214315  0.3766479   0.04613325  0.53248446  0.85821007
  0.49999026  0.73913991  0.57991674  0.05103247]
const    1.0
x1       5.0
x2       2.0
x1:x2    4.0
dtype: float64
(100, 4)
